In [7]:
import hashlib
import requests
import json
import pymongo
from pymongo import MongoClient
import logging
import threading
import time
agent_account = 'd79bd00d-ca25-4167-96b8-31c4a25778e1'
app_key = '126e7e38-ef8c-427f-8387-90202c9721e4'
api_server = 'http://www.lvcang.cn'
client=MongoClient('mongodb://localhost:27017')
db=client.test
scenic=db.menpiao_scenic_tongcheng
goods=db.menpiao_goods_tongcheng

## 查询各地区景区列表接口

In [128]:
def sce_post(i):
    #MD5
    sign = hashlib.md5(
        '{}{}{}{}'.format(agent_account, 20, i, app_key).encode('utf-8')
    ).hexdigest()
    #data
    data={
        "requestHead": {
            "digitalSign": sign,
            "agentAccount": agent_account
        },
        "requestBody": {
            "pageSize": 20,
            "pageIndex": i
        }
    }
    #post 第 i 页
    r=requests.post('http://www.lvcang.cn/jingqu/Services/GetSceneryListService.ashx',data=json.dumps(data))
    return r
def sce_for_insert(i):
    #调用post函数，创建r对象
    r=sce_post(i)
    #遍历i页条目，并插入mongoDB
    N=r.json()['TotalCount']
    n=len(r.json()['SceneryList'])
    for i_l in range(n):
        item = r.json()['SceneryList'][i_l]
        len(item.keys())
        item_dict={
            'id': item['SceneryID'],
            'name':item['SceneryName'],
            "grade":item['SceneryGrade'],
            'address':item['SceneryAddress'],
            'province':item['SceneryProvinceName'],
            'city':item['SceneryCityName'],
            'summary':item['ScenerySummary'],
            'lng':item['SceneryLongitude'],
            'lat':item['SceneryLatitude'],
            'alias':item['SceneryAliasName'],
            'traffic':item['SceneryTrafficGuide'],
            'notes':item['UserNotes'],
            "introduce": item['SceneryDetailIntroduce'],
            "ticketType": item['TicketType'],
            "ticketTypeName": item['TicketTypeName'],
            "picture": item['PictureListInfo']
        }
        scenic.update_one({'id': item_dict['id']}, {'$set':item_dict}, upsert=True)
    print('>>>新增总数：%d,插入第%d页,本页插入数：%d'%(scenic.find({}).count(),i,n)) 
    #s=scenic.find().count()
#     if i==160:
#         o=1/0
    return N
def sce_del():
    sce_sum=scenic.find({}).count()
    scenic.delete_many({})
    print('<<<d删除结束d>>>，总计%d条记录'%(sce_sum))

In [143]:
import hashlib
import requests
import json
import pymongo
from pymongo import MongoClient
import logging
import math
agent_account = 'd79bd00d-ca25-4167-96b8-31c4a25778e1'
app_key = '126e7e38-ef8c-427f-8387-90202c9721e4'
api_server = 'http://www.lvcang.cn'
client=MongoClient('mongodb://localhost:27017')
db=client.test
scenic=db.menpiao_scenic_tongcheng
N=sce_for_insert(1)
n=math.ceil(N/20)
for i in range(2,n+1):
    try:
        t=threading.Thread(target=sce_for_insert,args=(i,))
        t.start()
    except Exception as e:
        logging.exception(e)

>>>新增总数：20,插入第1页,本页插入数：20
>>>新增总数：249,插入第2页,本页插入数：20>>>新增总数：274,插入第5页,本页插入数：20

>>>新增总数：293,插入第3页,本页插入数：20
>>>新增总数：361,插入第9页,本页插入数：20>>>新增总数：387,插入第12页,本页插入数：20

>>>新增总数：432,插入第4页,本页插入数：20
>>>新增总数：462,插入第8页,本页插入数：20>>>新增总数：459,插入第20页,本页插入数：20>>>新增总数：483,插入第16页,本页插入数：20>>>新增总数：484,插入第7页,本页插入数：20>>>新增总数：523,插入第11页,本页插入数：20>>>新增总数：512,插入第17页,本页插入数：20>>>新增总数：566,插入第10页,本页插入数：20>>>新增总数：643,插入第21页,本页插入数：20>>>新增总数：664,插入第15页,本页插入数：20>>>新增总数：633,插入第6页,本页插入数：20>>>新增总数：717,插入第29页,本页插入数：20>>>新增总数：691,插入第22页,本页插入数：20>>>新增总数：785,插入第43页,本页插入数：20




>>>新增总数：857,插入第24页,本页插入数：20






>>>新增总数：797,插入第13页,本页插入数：20>>>新增总数：835,插入第14页,本页插入数：20>>>新增总数：877,插入第31页,本页插入数：20>>>新增总数：914,插入第32页,本页插入数：20>>>新增总数：914,插入第27页,本页插入数：20>>>新增总数：965,插入第46页,本页插入数：20>>>新增总数：979,插入第30页,本页插入数：20>>>新增总数：1000,插入第35页,本页插入数：20>>>新增总数：936,插入第25页,本页插入数：20>>>新增总数：922,插入第39页,本页插入数：20>>>新增总数：1022,插入第19页,本页插入数：20>>>新增总数：1009,插入第28页,本页插入数：20>>>新增总数：1029,插入第37页,本页插入数：20>>>新增总数：1039,插入第54页,本页插入数：20>>>新增总数：1037,插入第34页,本页插入数：20>>>新增总数：1029,

## 各景区所属票型查询接口

In [112]:
#分销商票型查询接口
import hashlib
import requests
import json
import pymongo
from pymongo import MongoClient
import logging
import threading
agent_account = 'd79bd00d-ca25-4167-96b8-31c4a25778e1'
app_key = '126e7e38-ef8c-427f-8387-90202c9721e4'
api_server = 'http://www.lvcang.cn'
client=MongoClient('mongodb://localhost:27017')
db=client.test
scenic=db.menpiao_scenic_tongcheng
goods=db.menpiao_goods_tongcheng

def my_post(i,sce_id):
    #MD5
    sign = hashlib.md5(
        '{}{}{}{}'.format(agent_account, 20, i, app_key).encode('utf-8')
    ).hexdigest()
    #data
    data={
        "requestHead": {
        "digitalSign": sign,
        "agentAccount": agent_account
        },
        "requestBody": {
        "pageSize": 20,
        "pageIndex": i,
        "sceneryId": sce_id   #景点id
        }
}
    #post sce_id景点的第i页
    r=requests.post('http://www.lvcang.cn/jingqu/Services/GetTicketDetailService.ashx',data=json.dumps(data))
    resp_data = r.json()
    return resp_data.get('ticketPriceList', [])

def my_for_insert(i,sce_id):
    #调用post函数，创建r对象
    data = my_post(i,sce_id)
    #遍历i页条目，并插入mongoDB
    n=len(data)
    for i_l in range(n):
        item = data[i_l]
        item_dict=item
        for del_key in ['BCTTicketPriceMode','isCopSys','isPreference']:
            item_dict.pop(del_key,'')
        goods.update_one({'ticketPriceId':item_dict['ticketPriceId']},{'$set':item_dict},upsert=True)
    return n

def my_insert(ids):
    for sce_id in ids:    
        i=1   # iiiii初始为1
        while(i):
            n=0
            try:
                n=my_for_insert(i,sce_id)
            except Exception as e:
                logging.exception(e)
            print('>>>新增总数：%d,id:%d,第%d页,本页插入数：%d\n'%(goods.find().count(),sce_id,i,n))        
            if n==20: 
                i=i+1
            else:
                i=0

def goo_del():
    summ=goods.find({}).count()
    goods.delete_many({})
    print('<<<d删除结束d>>>，总计%d条记录'%(summ))

In [228]:
#多线程编码@@@
sce=scenic.find({},{'id':1,'_id':0})  #获取景点id
ids=[]
for item in sce:
    ids.append(item['id'])
for i in range(200):
    thread=threading.Thread(target=my_insert,args=(ids[i*30:i*30+30],))
    thread.start()
    threading.active_count()
#     thread.join()

>>>新增总数：9244,id:36,第1页,本页插入数：1

>>>新增总数：9244,id:2640,第1页,本页插入数：2

>>>新增总数：9244,id:1250,第1页,本页插入数：1

>>>新增总数：9244,id:3105,第1页,本页插入数：1

>>>新增总数：9244,id:2827,第1页,本页插入数：3

>>>新增总数：9244,id:23774,第1页,本页插入数：1

>>>新增总数：9244,id:10803,第1页,本页插入数：1

>>>新增总数：9244,id:24987,第1页,本页插入数：1

>>>新增总数：9244,id:8646,第1页,本页插入数：1
>>>新增总数：9244,id:17852,第1页,本页插入数：3
>>>新增总数：9244,id:4087,第1页,本页插入数：3



>>>新增总数：9244,id:12683,第1页,本页插入数：2

>>>新增总数：9244,id:12760,第1页,本页插入数：1

>>>新增总数：9244,id:8714,第1页,本页插入数：1

>>>新增总数：9244,id:3403,第1页,本页插入数：1
>>>新增总数：9244,id:7506,第1页,本页插入数：1


>>>新增总数：9244,id:5947,第1页,本页插入数：1
>>>新增总数：9244,id:18919,第1页,本页插入数：1
>>>新增总数：9244,id:7923,第1页,本页插入数：1
>>>新增总数：9244,id:26401,第1页,本页插入数：1
>>>新增总数：9244,id:9393,第1页,本页插入数：1
>>>新增总数：9244,id:3311,第1页,本页插入数：1
>>>新增总数：9244,id:26145,第1页,本页插入数：6
>>>新增总数：9244,id:27373,第1页,本页插入数：3
>>>新增总数：9244,id:17740,第1页,本页插入数：1








>>>新增总数：9244,id:27297,第1页,本页插入数：2
>>>新增总数：9244,id:26817,第1页,本页插入数：1
>>>新增总数：9244,id:28409,第1页,本页插入数：3
>>>新增总数：9244,id:10186,第1页,本页插入数：2
>>>新增总

## 各景区-票型价格3个月日历查询接口

In [37]:

import hashlib
import requests
import json
import pymongo
from pymongo import MongoClient
import logging
import threading
import math
import progressbar
agent_account = 'd79bd00d-ca25-4167-96b8-31c4a25778e1'
app_key = '126e7e38-ef8c-427f-8387-90202c9721e4'
api_server = 'http://www.lvcang.cn'
client=MongoClient('mongodb://localhost:27017')
db=client.test
scenic=db.menpiao_scenic_tongcheng
goods=db.menpiao_goods_tongcheng

def prices_post(priceid):
    #MD5
    sign = hashlib.md5(
        '{}{}{}'.format(agent_account,priceid, app_key).encode('utf-8')
    ).hexdigest()
    #data
    data={
        "requestHead": {
        "digitalSign": sign,
        "agentAccount": agent_account
        },
        "requestBody": {
        "PriceId": priceid,
        "TravelDate": "2016-12-20",
        "MonthNumber": 3
        }
    }
    #post price_id景点的第i页
    r=requests.post('http://www.lvcang.cn/jingqu/Services/GetPriceCalendarInfoService.ashx',data=json.dumps(data))
    resp_data = r.json()
    return resp_data.get('priceList', [])
def my_for_insert(priceid):
    #调用post函数，创建r对象
    data = prices_post(priceid)
    #遍历i页条目，并插入mongoDB
    goods.update_one({'ticketPriceId':priceid},{'$set':{'prices':data}},upsert=True)
def my_insert(ids):
    t=len(ids)
    for priceid in ids:    
        try:
            my_for_insert(priceid)
        except Exception as e:
            logging.exception(e)

def goo_del():
    summ=goods.find({}).count()
    goods.delete_many({})
    print('<<<d删除结束d>>>，总计%d条记录'%(summ))

In [38]:
#3个月票价--多线程编码@@@
good=goods.find({},{'ticketPriceId':1,'_id':0})  #获取景点ticketPriceId
ids=[]
for item in good:
    ids.append(item['ticketPriceId'])
p=progressbar.ProgressBar()
m=len(ids) #任务总数m
n=100 #开启线程数量
j=math.ceil(m/n) #平均每个线程分配任务数量j
for i in range(n):
    thread=threading.Thread(target=my_insert,args=(ids[i*j:i*j+j],))
    thread.start()
print('>>>Over')
#     threading.active_count()
# thread.join()


>>>Over


## 产品变更增量拉取借口

In [3]:
def increament_post(maxIncreament):
    sign = hashlib.md5(
        '{}{}{}{}'.format(agent_account,maxIncreament, 20, app_key).encode('utf-8')
    ).hexdigest()
    #data
    data={
        "requestHead": {
        "digitalSign": sign,
        "agentAccount": agent_account
        },
        "requestBody": {
        "maxIncrementId": maxIncreament,
        "queryNumber": 20
        }
    }
    #post sce_id景点的第i页
    r=requests.post('http://www.lvcang.cn/jingqu/Services/GetTicketIncrementService.ashx',data=json.dumps(data))
    resp_data = r.json()
    return resp_data.get('ticketPriceList', []),resp_data.get('maxIncrementId', [])

def increament_for_insert(inc):
    data,maxinc=increament_post(inc)
    if inc!=maxinc:
        thread=threading.Thread(target=increament_for_insert,args=(maxinc,))
        thread.start()
    n=len(data)
    for i_l in range(n):
        item = data[i_l]
        item_dict=item
        for del_key in ['BCTTicketPriceMode','isCopSys','isPreference']:
            item_dict.pop(del_key,'')
#         goods.update_one({'ticketPriceId':item_dict['ticketPriceId']},{'$set':item_dict},upsert=True)
        print('>>>更新进度maxIncrement:{}--i:{},启用进程数：{}>>>'.format(inc,i_l,threading.active_count()),end='\r')


def increament_start():
    try:
        increament_for_insert(12534871)
    except Exception as e:
        logging.exception(e)

In [3]:
%time
increament_start()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 12.9 µs
>>>更新进度maxIncrement:10933332--i:16,启用进程数：9>>>>xIncrement:0--i:2,启用进程数：8>>>>>>更新进度maxIncrement:0--i:4,启用进程数：8>>>>>>更新进度maxIncrement:0--i:6,启用进程数：8>>>>>>更新进度maxIncrement:0--i:8,启用进程数：8>>>>>>更新进度maxIncrement:0--i:10,启用进程数：8>>>>>>更新进度maxIncrement:0--i:12,启用进程数：8>>>>>>更新进度maxIncrement:0--i:14,启用进程数：8>>>>>>更新进度maxIncrement:0--i:16,启用进程数：8>>>>>>更新进度maxIncrement:0--i:18,启用进程数：8>>>>>>更新进度maxIncrement:10697973--i:0,启用进程数：9>>>>>>更新进度maxIncrement:10697973--i:2,启用进程数：9>>>>>>更新进度maxIncrement:10697973--i:4,启用进程数：9>>>>>>更新进度maxIncrement:10697973--i:6,启用进程数：9>>>>>>更新进度maxIncrement:10697973--i:8,启用进程数：9>>>>>>更新进度maxIncrement:10697973--i:10,启用进程数：9>>>>>>更新进度maxIncrement:10697973--i:12,启用进程数：9>>>>>>更新进度maxIncrement:10697973--i:14,启用进程数：9>>>>>>更新进度maxIncrement:10697973--i:16,启用进程数：9>>>>>>更新进度maxIncrement:10697973--i:18,启用进程数：9>>>>>>更新进度maxIncrement:10698588--i:0,启用进程数：9>>>>>>更新进度maxIncrement:10698588--i:2,启用进程数：9>>>>>>更新进度maxIncrement:10698588

In [32]:
#更新票价绑定更新所属景区信息
def scenery_fetch_one(sceneryid):
    #MD5
    sign = hashlib.md5(
        '{}{}{}'.format(agent_account, sceneryid, app_key).encode('utf-8')
    ).hexdigest()
    #data
    data={
        "requestHead": {
            "digitalSign": sign,
            "agentAccount": agent_account
        },
        "requestBody": {
            "sceneryId": sceneryid
        }}
    try:
        resp=requests.post('http://www.lvcang.cn/jingqu/Services/GetSceneryDetailService.ashx',data=json.dumps(data))
        item=resp.json()
    except Exception as e:
        logging.exception(e)
    item_dict={
        'id': item['SceneryID'],
        'name':item['SceneryName'],
        "grade":item['SceneryGrade'],
        'address':item['SceneryAddress'],
        'province':item['SceneryProvinceName'],
        'city':item['SceneryCityName'],
        'summary':item['ScenerySummary'],
        'lng':item['SceneryLongitude'],
        'lat':item['SceneryLatitude'],
        'alias':item['SceneryAliasName'],
        'traffic':item['SceneryTrafficGuide'],
        'notes':item['UserNotes'],
        "introduce": item['SceneryDetailIntroduce'],
        "ticketType": item['TicketType'],
        "ticketTypeName": item['TicketTypeName'],
        "picture": item['PictureListInfo']
    }
    scenic.update_one({'id': item_dict['id']}, {'$set':item_dict}, upsert=True)
    return item

In [33]:
datas = scenery_fetch_one(222776)
datas

{'PictureListInfo': [{'SceneryImgPath': 'http://pic3.40017.cn/scenery/destination/2015/09/29/18/9dNzjN_740x350_00.jpg',
   'SceneryImgRemarks': ''},
  {'SceneryImgPath': 'http://pic3.40017.cn/scenery/destination/2015/09/29/18/ylQekB_740x350_00.jpg',
   'SceneryImgRemarks': ''},
  {'SceneryImgPath': 'http://pic3.40017.cn/scenery/destination/2015/09/29/18/MNOUYK_740x350_00.jpg',
   'SceneryImgRemarks': ''},
  {'SceneryImgPath': 'http://pic3.40017.cn/scenery/destination/2015/09/29/18/WsbyE6_740x350_00.jpg',
   'SceneryImgRemarks': ''},
  {'SceneryImgPath': 'http://pic3.40017.cn/scenery/destination/2015/09/29/18/Cs4yuv_740x350_00.jpg',
   'SceneryImgRemarks': ''},
  {'SceneryImgPath': 'http://pic3.40017.cn/scenery/destination/2015/09/29/18/m3fRR8_740x350_00.jpg',
   'SceneryImgRemarks': ''}],
 'SceneryAddress': '河南省南阳市卧龙区卧龙路766号',
 'SceneryAliasName': '',
 'SceneryCityName': '南阳',
 'SceneryDetailIntroduce': '<h4 class="intro_head">去<font>南阳卧龙岗武侯祠</font>的N大理由</h4>\r\n<!-- 简介模块头部 -->\r\n<ul 

In [30]:
datas

222776

In [ ]:
N=sce_for_insert(1)
n=math.ceil(N/20)
for i in range(2,n+1):
    try:
        t=threading.Thread(target=sce_for_insert,args=(i,))
        t.start()
    except Exception as e:
        logging.exception(e)

In [183]:
data['date'] < '2016-12-31'
arr=['2016-11-26','2016-11-27','2016-11-28','2016-11-30','2016-11-31','2016-12-01']
t='2016-11-29'
arr_index=BinarySearch(arr,t)
arr_index
# arr[2]<t

TypeError: string indices must be integers

In [269]:
import math
def BinarySearch(array,t):

    low = 0
    height = len(array)-1
    while low <= height:
        mid = math.floor((low+height)/2)

        if len(array[mid]['date'])!=10:
            array[mid]['date']=time.mktime(time.strptime(array[mid]['date'], '%Y-%m-%d'))
            t=time.mktime(time.strptime(t, '%Y-%m-%d'))

        if array[mid]['date'] < t:
            low = mid + 1

        elif array[mid]['date'] > t:
            height = mid - 1

        else:
            return mid
        
    return low
def remove_old_prices(data_price):
    if len(data_price.get('prices',[]))!=0:
        n = BinarySearch(data_price['prices'],today_date)
        data_price['prices'] = data_price['prices'][n:]
        goods.update_one({'ticketPriceId': data_price['ticketPriceId']}, {'$set':data_price})

In [270]:
import time
import progressbar
p = progressbar.ProgressBar()
today_date = time.strftime('%Y-%m-%d', time.localtime(time.time()))
data_prices=goods.find({},projection={'prices':1,'ticketPriceId':1,'_id':0})

for data_price in p(data_prices):
    remove_old_prices(data_price)
print('over')

| 10698 Elapsed Time: 0:00:46                                                  - 1 Elapsed Time: 0:00:00                                                      


over


In [259]:
a=[1]
a[1:]

[]

In [244]:
n=BinarySearch(data_price['prices'],'2016-12-27')
# data_price['prices']
data_price['prices']=data_price['prices'][n[0]:]
data_price['prices']



[{'agentAmount': 72,
  'date': '2016-12-27',
  'isStock': 0,
  'marketAmount': 80,
  'maxTickets': 99,
  'minTickets': 1,
  'priceId': 378076,
  'stockNum': None,
  'tcAmount': 75,
  'ticketName': '伪满皇宫博物院成人票'},
 {'agentAmount': 72,
  'date': '2016-12-28',
  'isStock': 0,
  'marketAmount': 80,
  'maxTickets': 99,
  'minTickets': 1,
  'priceId': 378076,
  'stockNum': None,
  'tcAmount': 75,
  'ticketName': '伪满皇宫博物院成人票'},
 {'agentAmount': 72,
  'date': '2016-12-29',
  'isStock': 0,
  'marketAmount': 80,
  'maxTickets': 99,
  'minTickets': 1,
  'priceId': 378076,
  'stockNum': None,
  'tcAmount': 75,
  'ticketName': '伪满皇宫博物院成人票'},
 {'agentAmount': 72,
  'date': '2016-12-30',
  'isStock': 0,
  'marketAmount': 80,
  'maxTickets': 99,
  'minTickets': 1,
  'priceId': 378076,
  'stockNum': None,
  'tcAmount': 75,
  'ticketName': '伪满皇宫博物院成人票'},
 {'agentAmount': 72,
  'date': '2016-12-31',
  'isStock': 0,
  'marketAmount': 80,
  'maxTickets': 99,
  'minTickets': 1,
  'priceId': 378076,
  'stockN